In [10]:
import pandas as pd
import numpy as np
import ast
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer


<h2>Cargar el csv y regular algunas columnas</h2>

In [11]:
#cargamos en "data"
data= pd.read_csv("G:\Mi unidad\SOYHENRY\CIENCIA DE DATOS\CURSO\PROYECTOS\PI1_ML\datasetLimpio\datasetLimpio.csv")

In [12]:
#regulamos columnas que eran listas
#definimos la función que transformará en lista todos aquellos datos que perdieron esta condición
def obtener(celda):
    if pd.isnull(celda):
        return None
    if isinstance(celda, str) and celda.startswith("[") and celda.endswith("]"):
        try:
            return ast.literal_eval(celda)
        except (ValueError, SyntaxError):
            return celda  # Devuelve la celda original si no se puede convertir a lista
    return celda

#aplicamos esta función a todo el dataframe
data = data.applymap(obtener)

<h2>Función para quitar los stopwords</h2>

In [13]:
nltk.download('punkt')  # Descargar el tokenizer
nltk.download('stopwords')  # Descargar las palabras vacías

#función para quitar los stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jhose\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jhose\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
data.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'revenue', 'runtime', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'release_year', 'release_month', 'release_day', 'return',
       'nameBTC', 'name_genre', 'name_companie', 'pcountry', 'language',
       'Director'],
      dtype='object')

<h2>Matriz de entranamiento con "TfidfVectorizer"</h2>

In [17]:

# Combinar las columnas en una nueva columna 'combinacion'
data['combinacion'] = (
    data['overview'] +
    ' ' +
    data['name_genre'].apply(lambda x: ' '.join(x) if isinstance(x, list) else '') +
    ' ' +
    data['nameBTC'].apply(lambda x: ' '.join(''.join(name.split()) for name in x) if isinstance(x, list) else '') +
    ' ' +
    data['Director'].apply(lambda x: ''.join(x.split()) if isinstance(x, str) else '') +
    ' ' +
    data['popularity'].astype(str) +
    ' ' +
    data['vote_average'].astype(str)
)

# Reemplazar NaN con cadenas vacías en la columna 'combinacion'
data['combinacion'] = data['combinacion'].fillna('')

# Crear una instancia de TfidfVectorizer para vectorizar el texto combinado
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))

# Aplicar TF-IDF al texto combinado
matrix = tfidf.fit_transform(data['combinacion'])


In [18]:
def recomendacionKNN(title):
    # Crear una instancia del clasificador KNN
    knn = NearestNeighbors(n_neighbors=6, metric='cosine')

    # Ajustar el modelo KNN utilizando la matriz tfidf_matrix
    knn.fit(matrix)

    # Encontrar el índice de la película de entrada
    entrada_index = data[data['title'] == title].index[0]

    # Encontrar los vecinos más cercanos a la película de entrada
    distances, indices = knn.kneighbors(matrix[entrada_index])

    # Recuperar los títulos de las películas recomendadas utilizando los índices encontrados
    recommended_movies = data.iloc[indices[0][1:]]['title']

    # Devolver las películas recomendadas
    return recommended_movies


In [19]:
#probamos el modelo
recomendacionKNN('Toy Story')

2997                     Toy Story 2
15350                    Toy Story 3
10303         The 40 Year Old Virgin
23816    Andy Hardy's Blonde Trouble
24495                      Small Fry
Name: title, dtype: object